In [13]:
!pip install -q lion-pytorch comet_ml

In [14]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
import torch.nn.functional as F
import torchvision.transforms as T

from PIL import Image
import json
from math import prod
from huggingface_hub import hf_hub_download
import tqdm
transform_t_to_pil = T.ToPILImage()
transform_to_t = transforms.Compose([transforms.ToTensor()])
import utils
from utils import get_all_bounding_boxes, segment, calculate_initial_theta, stack_alpha_aware, AestheticPredictor, get_random_initial_position
# from lion_pytorch import Lion
from datetime import datetime
import random
from huggingface_hub import hf_hub_download
import timm
from lion_pytorch import Lion
import torch.nn as nn
import comet_ml

In [3]:
# Transformations
resnet_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [4]:
device = "cpu"

In [5]:
# Load pre-trained ResNet model
random_predictor = timm.create_model('resnet50', pretrained=True)
random_predictor.fc = nn.Linear(random_predictor.fc.in_features, 2)  # Modify the classifier to output 2 classes
random_predictor.to(device)

local_filename = hf_hub_download(repo_id="mowoe/random-layout-classifier", filename="random-layout-classifier.pt")
random_predictor.load_state_dict(torch.load(local_filename, map_location="cpu"))

<All keys matched successfully>

In [6]:
im = Image.open('../combined/5373.jpg')
im = im.convert('RGBA')
im = im.resize((1440, 2560), Image.Resampling.LANCZOS)


with open('../combined/5373.json', "r") as f:
    image_json = json.load(f)

reduced_segments = [s for s in segment(im,image_json) if (prod(s[0].size) < 0.80*prod(im.size)) and (prod(s[0].size)>1)]

In [7]:
im.size

(1440, 2560)

In [8]:
def get_all_bounding_boxes(item):
    all_boxes = []
    if "bounds" in item.keys() and "resource-id" in item.keys():
        all_boxes.append((item["bounds"],item["resource-id"]))
    if "children" in item.keys():
        for child in item["children"]:
            for box in get_all_bounding_boxes(child):
                all_boxes.append(box)
    return all_boxes

In [9]:
class MultiSegmentUIOptimizerWithBackgroundParam(torch.nn.Module):
    def __init__(self,segments_and_positions: list, original_image_size: tuple):
        super().__init__()
        self.original_image_size = original_image_size
        self.segments_and_positions = segments_and_positions
        self.canvas_size = (1,3,original_image_size[1],original_image_size[0])
        print(f"Canvas size is {self.canvas_size}")

        initial_vector = []
        for segment,position in segments_and_positions:
            # initial_theta = calculate_initial_theta(segment,original_image_size,position)
            initial_theta = get_random_initial_position(segment,original_image_size,position)
            initial_vector.append([initial_theta[0][2],initial_theta[1][2]])

        self.coordinates = torch.nn.Parameter(torch.tensor(initial_vector))
        print(self.coordinates)
        self.background_color = torch.nn.Parameter(torch.tensor(np.array([1.0,1.0,1.0]),dtype=torch.float))
        

    def forward(self):
        segments_on_canvas = []

        # Create background image from parameter
        bg_col = torch.clamp(self.background_color, min=0, max=1)
        #print(f"Clamped bg to {bg_col}")
        red = torch.tile(bg_col[0],self.original_image_size[::-1])
        green = torch.tile(bg_col[1],self.original_image_size[::-1])
        blue = torch.tile(bg_col[2],self.original_image_size[::-1])
        alpha = torch.tile(torch.tensor(1.0),self.original_image_size[::-1])
        background = torch.stack([red,green,blue,alpha]).unsqueeze(0)
        #print(background.detach().sum())
        #background = torch.tile(self.background_color,self.original_image_size)
        
        segments_on_canvas.append(background)
        
        for n in range(len(self.segments_and_positions)):
            # We need to calculate the proper ratios, to artificially warp the segment on to a bigger canvas without distorting it (see notebook 01)
            x_ratio = self.original_image_size[0] / self.segments_and_positions[n][0].size[0]
            y_ratio = self.original_image_size[1] / self.segments_and_positions[n][0].size[1]
            
            # Affine matrix
            theta = [
                [x_ratio, 0.0    , 0.0],
                [0.0    , y_ratio, 0.0]
            ]
            theta_tensor = torch.as_tensor(theta)[None]

            theta_tensor[0][0][2]+=self.coordinates[n][0]
            theta_tensor[0][1][2]+=self.coordinates[n][1]
            
            # Generate flow field
            grid = F.affine_grid(theta_tensor, self.canvas_size).type(torch.FloatTensor)
            x = F.grid_sample(transform_to_t(self.segments_and_positions[n][0]).unsqueeze(0), grid)
            segments_on_canvas.append(x)



        return segments_on_canvas

In [15]:
appsthetics_predictor = torch.load("../Dataset_estetica/Regressao/model123")

In [11]:
learning_rates = [0.1,0.05,0.01,0.005,0.001,0.0005]
optimisers = [Lion,torch.optim.AdamW]

In [22]:
random_classifier_criterion = nn.CrossEntropyLoss()
# random_classifier_loss_weight = 0.33
random_classifier_loss_weight = 1.0
random_classifier_target = torch.Tensor([1.0]).to(torch.long)

In [ ]:
for lr in learning_rates:
    for optimiser in optimisers:
        experiment = comet_ml.Experiment(
            api_key="<revoked>",
            project_name="mowoe/bachelor-thesis-hyperparam-sweep"
        )

        experiment.log_parameters({
            "optim":str(optimiser),
            "lr":lr
        })

        reduced_losses = []

        # Try for 3 different seeds: 
        for y in range(3):
            msUIo_wbg = MultiSegmentUIOptimizerWithBackgroundParam(reduced_segments, im.size)
            
            
            optim = optimiser(msUIo_wbg.parameters(),lr=lr)
            initial_loss = None
    
            
            
            for x in tqdm.tqdm(range(25)):
                optim.zero_grad()
    
                canvasses = msUIo_wbg()
        
                generated_image = stack_alpha_aware(canvasses)
                # Remove Alpha channel
                generated_image = generated_image[:3]
            
                score = appsthetics_predictor["model"](generated_image.unsqueeze(0))
            
                random_classifier_logits = random_predictor(resnet_transform(generated_image.unsqueeze(0)))
                random_loss = random_classifier_criterion(random_classifier_logits,random_classifier_target)
            
                combined_loss = random_classifier_loss_weight * random_loss + (1-random_classifier_loss_weight) * ((1-score)*10)
                    
                loss = -score
                loss.backward()
                optim.step()
    
                if not initial_loss:
                    initial_loss = combined_loss.item()
               
            
                # Save image as png to create gif
                pil_image = transform_t_to_pil(generated_image.detach())
                pil_image = pil_image.resize((int(pil_image.size[0]/4),int(pil_image.size[1]/4)), Image.Resampling.LANCZOS)
                experiment.log_image(pil_image,step=(x+10*y))
                experiment.log_metric("loss", combined_loss.item(), step=(x+10*y))
        
        reduced_losses.append((1-(combined_loss.item()/initial_loss))**2)

        

        experiment.log_metric("avg_squared_loss_perc_reduction",sum(reduced_losses)/len(reduced_losses))
        
        experiment.end()

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : dull_barbel_5024
COMET INFO:     url                   : https://www.comet.com/mowoe/mowoe-bachelor-thesis-hyperparam-sweep/ab83a8b13e7145d48c2a583aa1ca46e7
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [24] : (7.164503574371338, 10.778894424438477)
COMET INFO:   Parameters:
COMET INFO:     lr    : 0.1
COMET INFO:     optim : <class 'torch.optim.adamw.AdamW'>
COMET INFO:   Uploads:
COMET INFO:     conda-environment-definition : 1
COMET INFO:     conda-info                   : 1
COMET INFO:     conda-specification          : 1
COM

Canvas size is (1, 3, 2560, 1440)
Parameter containing:
tensor([[ -1.9200, -10.6559],
        [ -0.3738,  -6.4977],
        [  0.3894,  -0.8312],
        [  0.0752,  19.9029],
        [  2.6668,   2.4214],
        [  0.0000, -12.8590],
        [  0.0000,   5.7405]], dtype=torch.float64, requires_grad=True)


  0%|          | 0/25 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/torch/nn/functional.py:4404: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/torch/nn/functional.py:4343: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
  4%|▍         | 1/25 [00:14<05:44, 14.37s/it]/opt/conda/lib/python3.11/site-packages/torch/nn/functional.py:4404: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warning

Canvas size is (1, 3, 2560, 1440)
Parameter containing:
tensor([[  5.1258,  -4.2257],
        [  0.4999, -10.3483],
        [  0.1464,  -7.7551],
        [ -1.9631, -24.7587],
        [  3.1382,  -2.9370],
        [  0.0000,   7.3928],
        [  0.0000,  19.6109]], dtype=torch.float64, requires_grad=True)


100%|██████████| 25/25 [05:50<00:00, 14.02s/it]


...

100%|██████████| 25/25 [03:47<00:00,  9.09s/it]


Canvas size is (1, 3, 2560, 1440)
Parameter containing:
tensor([[  4.8931,   0.2500],
        [ -0.6618, -12.7307],
        [ -0.4186,   7.0656],
        [  0.1232,  -4.4600],
        [ -1.7523,  11.7416],
        [  0.0000,   5.1870],
        [  0.0000,  -8.2909]], dtype=torch.float64, requires_grad=True)


100%|██████████| 25/25 [03:47<00:00,  9.10s/it]
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : different_tick_8668
COMET INFO:     url                   : https://www.comet.com/mowoe/mowoe-bachelor-thesis-hyperparam-sweep/3c084967d7014c0cab4582e79d8b3466
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     avg_squared_loss_perc_reduction : 0.013393714434265309
COMET INFO:     loss [75]                       : (5.4227519035339355, 8.103497505187988)
COMET INFO:   Parameters:
COMET INFO:     lr    : 0.005
COMET INFO:     optim : <class 'torch.optim.adamw.AdamW'>
COMET INFO:   Uploads:
COMET INFO:     conda-environment-definition : 1
COMET INFO:     conda-info                   : 1
COMET INFO: 

Canvas size is (1, 3, 2560, 1440)
Parameter containing:
tensor([[  1.8563,  -5.8283],
        [  0.6481,   0.7591],
        [  0.5925,   7.2506],
        [  1.6428, -25.5575],
        [ -1.0654,  -5.5741],
        [  0.0000,   1.1248],
        [  0.0000, -20.5481]], dtype=torch.float64, requires_grad=True)


  0%|          | 0/25 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/torch/nn/functional.py:4404: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/torch/nn/functional.py:4343: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
  4%|▍         | 1/25 [00:08<03:27,  8.63s/it]/opt/conda/lib/python3.11/site-packages/torch/nn/functional.py:4404: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warning